In [4]:
import pandas as pd
import numpy as np
import plotly.express as px

# CSV読み込み
df = pd.read_csv("earthquakes.csv")

# 日本時間へ変換
df["time"] = pd.to_datetime(df["time"])
df["time_jst"] = df["time"] + pd.Timedelta(hours=9)

# 3時間ごとに丸める
df_anim = df.copy()
df_anim["time_3h"] = df_anim["time_jst"].dt.floor("3H")
df_anim["time_str"] = df_anim["time_3h"].dt.strftime("%Y-%m-%d %H:00")

# 深さで色分け
conditions = [
    (df_anim["depth"] <= 70),
    (df_anim["depth"] > 70) & (df_anim["depth"] <= 300),
    (df_anim["depth"] > 300)
]

colors = ["green", "orange", "blue"]  # 浅・中・深

df_anim["depth_color"] = np.select(conditions, colors, default="gray")

# マグニチュードでサイズ
df_anim["size"] = df_anim["mag"] ** 2


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\2687732498.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_anim["time_3h"] = df_anim["time_jst"].dt.floor("3H")


In [62]:
import pandas as pd
import requests

# USGSから過去1か月の地震データを取得
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"
response = requests.get(url)

# データをファイルに保存
with open('earthquakes.csv', 'wb') as file:
    file.write(response.content)

# データをDataFrameに読み込む
data = pd.read_csv('earthquakes.csv')

# データの最初の5行を表示
data.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2025-11-19T02:14:34.839Z,59.385216,-138.712936,5.815022,2.769645,ml,15.0,186.461739,0.450602,0.807060,...,2025-11-19T02:16:45.431Z,"60 km ESE of Yakutat, Alaska",earthquake,0.000000,2.748861,0.000000,1.0,automatic,ak,ak
1,2025-11-19T02:08:16.932Z,62.943947,-150.472092,103.293152,2.039515,ml,25.0,45.500549,0.441669,0.488629,...,2025-11-19T02:09:41.617Z,"52 km NNE of Petersville, Alaska",earthquake,0.000000,3.418342,0.479709,2.0,automatic,ak,ak
2,2025-11-19T01:52:37.540Z,37.762333,-121.943001,6.170000,1.790000,md,14.0,94.000000,0.025990,0.110000,...,2025-11-19T02:17:18.670Z,"4 km ESE of San Ramon, CA",earthquake,0.340000,0.660000,0.120000,12.0,automatic,nc,nc
3,2025-11-19T01:48:36.220Z,37.286167,-121.658501,5.660000,2.960000,md,89.0,41.000000,0.029530,0.070000,...,2025-11-19T02:30:16.668Z,"16 km E of Seven Trees, CA",earthquake,0.110000,0.230000,0.150000,132.0,automatic,nc,nc
4,2025-11-19T01:38:54.976Z,31.658000,-104.322000,7.150900,1.700000,ml,31.0,68.000000,0.100000,0.100000,...,2025-11-19T02:17:10.155Z,"57 km S of Whites City, New Mexico",earthquake,0.622503,0.725843,0.100000,18.0,reviewed,tx,tx


In [63]:
# Jupyter 上で実行
!pip install pandas requests plotly



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
import pandas as pd
import requests
from io import StringIO
from datetime import datetime, timedelta

# ------------------
# 期間を設定（過去1年）
# ------------------
end = datetime.utcnow()
start = end - timedelta(days=365)

# 月ごとに分割
def month_ranges(start, end):
    ranges = []
    cur = start.replace(day=1)
    while cur < end:
        if cur.month == 12:
            nxt = cur.replace(year=cur.year+1, month=1, day=1)
        else:
            nxt = cur.replace(month=cur.month+1, day=1)
        ranges.append((cur, min(nxt, end)))
        cur = nxt
    return ranges

ranges = month_ranges(start, end)

# ------------------
# USGS API
# ------------------
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

all_df = []

print("Requesting monthly USGS data...")

for s, e in ranges:
    params = {
        "format": "csv",
        "starttime": s.strftime("%Y-%m-%d"),
        "endtime": e.strftime("%Y-%m-%d"),
        "minmagnitude": 2.5,
        "orderby": "time"
    }
    print(f" → {params['starttime']} ～ {params['endtime']}")

    resp = requests.get(url, params=params)
    resp.raise_for_status()  # ← 500 が出たらここで止まる

    df = pd.read_csv(StringIO(resp.text))
    all_df.append(df)

# ------------------
# データを結合
# ------------------
df_all = pd.concat(all_df, ignore_index=True)

print(f"\nDONE! 総イベント数: {len(df_all)} 件")

# ------------------
# 日本時間（JST）に変換
# ------------------
df_all["time_jst"] = pd.to_datetime(df_all["time"], utc=True).dt.tz_convert("Asia/Tokyo")

df_all.head()


C:\Users\asami\AppData\Local\Temp\ipykernel_3540\101846607.py:9: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).



Requesting monthly USGS data...
 → 2024-11-01 ～ 2024-12-01
 → 2024-12-01 ～ 2025-01-01
 → 2025-01-01 ～ 2025-02-01
 → 2025-02-01 ～ 2025-03-01
 → 2025-03-01 ～ 2025-04-01
 → 2025-04-01 ～ 2025-05-01
 → 2025-05-01 ～ 2025-06-01
 → 2025-06-01 ～ 2025-07-01
 → 2025-07-01 ～ 2025-08-01
 → 2025-08-01 ～ 2025-09-01
 → 2025-09-01 ～ 2025-10-01
 → 2025-10-01 ～ 2025-11-01
 → 2025-11-01 ～ 2025-11-19

DONE! 総イベント数: 29291 件


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,time_jst
0,2024-11-30T23:52:45.796Z,-7.657000,127.755500,168.706,4.20,mb,19.0,91.0,3.777000,0.93,...,"126 km NE of Lospalos, Timor Leste",earthquake,8.64,8.252,0.132000,16.0,reviewed,us,us,2024-12-01 08:52:45.796000+09:00
1,2024-11-30T23:19:38.580Z,19.386167,-155.242833,1.960,3.08,ml,28.0,67.0,0.008029,0.11,...,"6 km S of Volcano, Hawaii",earthquake,0.17,0.150,0.397188,22.0,reviewed,hv,hv,2024-12-01 08:19:38.580000+09:00
2,2024-11-30T23:18:42.830Z,19.387667,-155.242833,1.810,2.50,ml,49.0,52.0,0.008285,0.10,...,"6 km S of Volcano, Hawaii",earthquake,0.19,0.180,0.262802,32.0,reviewed,hv,hv,2024-12-01 08:18:42.830000+09:00
3,2024-11-30T22:12:26.968Z,-22.581700,-12.956900,10.000,4.80,mb,42.0,50.0,26.676000,1.01,...,southern Mid-Atlantic Ridge,earthquake,12.03,1.884,0.089000,39.0,reviewed,us,us,2024-12-01 07:12:26.968000+09:00
4,2024-11-30T21:36:05.908Z,13.054300,143.926400,132.912,4.70,mb,150.0,40.0,1.060000,0.66,...,"83 km WSW of Merizo Village, Guam",earthquake,9.23,5.913,0.049000,129.0,reviewed,us,us,2024-12-01 06:36:05.908000+09:00


In [110]:
import requests
import pandas as pd
from io import StringIO
from datetime import datetime, timedelta, timezone

# ======================================
# ① 取得期間を設定（過去7日間）
# ======================================
end_time = datetime.now(timezone.utc)
start_time = end_time - timedelta(days=7)

# USGS API
url = "https://earthquake.usgs.gov/fdsnws/event/1/query"
params = {
    "format": "csv",
    "starttime": start_time.strftime("%Y-%m-%dT%H:%M:%S"),
    "endtime": end_time.strftime("%Y-%m-%dT%H:%M:%S"),
    "minmagnitude": 2.5
}

print("Requesting USGS data ...")
resp = requests.get(url, params=params)
resp.raise_for_status()

# CSV を pandas に読み込む
df = pd.read_csv(StringIO(resp.text))
print(f"Raw events fetched: {len(df)}")

# query.csv に保存
df.to_csv("query.csv", index=False)
print("Saved to query.csv")


Requesting USGS data ...
Raw events fetched: 247
Saved to query.csv


In [111]:
import pandas as pd

# query.csv を読み込み
df = pd.read_csv("query.csv")
print("CSV loaded:", df.shape)

# UTC → JST に変換
df["time_jst"] = (
    pd.to_datetime(df["time"], utc=True)
      .dt.tz_convert("Asia/Tokyo")
)

print(df[["time", "time_jst", "latitude", "longitude", "mag"]].head())


CSV loaded: (247, 22)
                       time                         time_jst   latitude  \
0  2025-11-19T02:45:00.925Z 2025-11-19 11:45:00.925000+09:00  55.460300   
1  2025-11-19T02:23:19.189Z 2025-11-19 11:23:19.189000+09:00  39.235200   
2  2025-11-19T02:14:34.839Z 2025-11-19 11:14:34.839000+09:00  59.385216   
3  2025-11-19T01:48:36.220Z 2025-11-19 10:48:36.220000+09:00  37.286167   
4  2025-11-19T01:38:53.994Z 2025-11-19 10:38:53.994000+09:00  10.870900   

    longitude       mag  
0 -157.214157  2.547544  
1  143.467800  5.000000  
2 -138.712936  2.769645  
3 -121.658501  2.960000  
4  -62.756600  5.000000  


In [112]:
pip install pandas plotly requests


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [117]:
import numpy as np

# 深さカテゴリ
conditions = [
    (df["depth"] < 10),
    (df["depth"] >= 10) & (df["depth"] < 50),
    (df["depth"] >= 50)
]

choices = ["浅い(Shallow)", "中間(Middle)", "深い(Deep)"]

df["depth_category"] = np.select(
    conditions,
    choices,
    default="不明(Unknown)"
)


import json

records = df.to_dict(orient="records")

with open("earthquake_data.json", "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False)

import plotly.express as px

fig = px.scatter_geo(
    df,
    lon="longitude",
    lat="latitude",
    color="depth_category",
    size="mag",
    animation_frame="date_jst",
    hover_name="time_jst",
    projection="natural earth",
    title="過去30日間の地震アニメーション（JST）"
)

fig.update_layout(
    legend_title="深さカテゴリ",
    geo=dict(
        showland=True,
        landcolor="rgb(229, 229, 229)"
    )
)

fig.show()



TypeError: Object of type Timestamp is not JSON serializable

In [118]:
import json
import numpy as np

# ① JST を文字列化（datetime は JSON にできない）
df["time"] = df["time"].astype(str)
df["time_jst"] = df["time_jst"].astype(str)
df["date_jst"] = df["date_jst"].astype(str)

# ② numpy 型 → Python ネイティブ型に変換
df = df.astype({
    "latitude": float,
    "longitude": float,
    "depth": float,
    "mag": float
})

# カテゴリも文字列確定
df["depth_category"] = df["depth_category"].astype(str)

# ③ 辞書化 → JSON 書き出し
records = df.to_dict(orient="records")

with open("earthquake_data.json", "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False)


KeyError: 'date_jst'

In [134]:
import pandas as pd

# CSV を読み込み
df = pd.read_csv("query.csv")
print("CSV loaded:", df.shape)

# UTC → JST に変換
df["time_jst"] = (
    pd.to_datetime(df["time"], utc=True)
      .dt.tz_convert("Asia/Tokyo")
)

# ← これが必要！
df["date_jst"] = df["time_jst"].dt.date

print(df[["time", "time_jst", "date_jst", "latitude", "longitude", "mag"]].head())


CSV loaded: (247, 22)
                       time                         time_jst    date_jst  \
0  2025-11-19T02:45:00.925Z 2025-11-19 11:45:00.925000+09:00  2025-11-19   
1  2025-11-19T02:23:19.189Z 2025-11-19 11:23:19.189000+09:00  2025-11-19   
2  2025-11-19T02:14:34.839Z 2025-11-19 11:14:34.839000+09:00  2025-11-19   
3  2025-11-19T01:48:36.220Z 2025-11-19 10:48:36.220000+09:00  2025-11-19   
4  2025-11-19T01:38:53.994Z 2025-11-19 10:38:53.994000+09:00  2025-11-19   

    latitude   longitude       mag  
0  55.460300 -157.214157  2.547544  
1  39.235200  143.467800  5.000000  
2  59.385216 -138.712936  2.769645  
3  37.286167 -121.658501  2.960000  
4  10.870900  -62.756600  5.000000  


In [135]:
import folium
from folium.plugins import MarkerCluster

# 地図の中心（適当に日本周辺に設定、必要あれば変更）
m = folium.Map(location=[35, 140], zoom_start=3)

# MarkerCluster（点が多いのでまとめる）
cluster = MarkerCluster().add_to(m)

# 地震をマップに表示
for _, row in df.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    mag = row["mag"]
    time = row["time_jst"]

    folium.CircleMarker(
        location=[lat, lon],
        radius=max(mag, 1),  # マグニチュードで大きさ変える
        color="red",
        fill=True,
        fill_opacity=0.7,
        popup=f"M {mag}<br>{time}"
    ).add_to(cluster)

# HTMLに保存
m.save("earthquakes_map.html")
print("✔ 地図を保存しました → earthquakes_map.html")


✔ 地図を保存しました → earthquakes_map.html


In [136]:
import folium
from folium.plugins import MarkerCluster
import webbrowser
import os

# 地図の中心
m = folium.Map(location=[35, 140], zoom_start=3)

# マーカーをまとめる
cluster = MarkerCluster().add_to(m)

# 地震をマップに表示
for _, row in df.iterrows():
    lat = row["latitude"]
    lon = row["longitude"]
    mag = row["mag"]
    time = row["time_jst"]

    folium.CircleMarker(
        location=[lat, lon],
        radius=max(mag, 1),
        color="red",
        fill=True,
        fill_opacity=0.7,
        popup=f"M {mag}<br>{time}"
    ).add_to(cluster)

# 保存
filename = "earthquakes_map.html"
m.save(filename)
print("✔ 地図ファイル作成:", filename)

# ブラウザで自動オープン
fullpath = os.path.abspath(filename)
webbrowser.open(f"file://{fullpath}")
print("🌏 自動でブラウザを開きました")



✔ 地図ファイル作成: earthquakes_map.html
🌏 自動でブラウザを開きました
